In [ ]:

#StudyMentor – Multi-Agent Exam Helper
# Kaggle-ready single cell code
# Requires: GOOGLE_API_KEY in Kaggle Secrets


!pip install --upgrade google-generativeai pypdf -q

import os
import google.generativeai as genai
from pypdf import PdfReader
from kaggle_secrets import UserSecretsClient

                          #GEMINI CONFIG 
user_secrets = UserSecretsClient()
API_KEY = user_secrets.get_secret("GOOGLE_API_KEY")

if not API_KEY:
    raise ValueError("❌ GOOGLE_API_KEY not found in environment. Set it in Kaggle 'Add-ons → Secrets'.")

genai.configure(api_key=API_KEY)
MODEL_NAME = "gemini-2.5-flash"


def llm(prompt: str, max_tokens: int = 300) -> str:
    model = genai.GenerativeModel(MODEL_NAME)  #  use global model name
    resp = model.generate_content(prompt)
    try:
        return resp.text
    except:
        return "⚠ Model refused to respond due to safety or access limitations."

                     #GLOBAL STATE 
pdf_text = None          # full text from loaded PDF
pdf_path_global = None   # last loaded pdf path


                      # UTILITY FUNCTIONS 
def read_pdf(path: str) -> str:
    """Read all text from a PDF file."""
    reader = PdfReader(path)
    pages = [page.extract_text() or "" for page in reader.pages]
    return "\n".join(pages)


def chunk_text(text: str, max_chars: int = 8000):
    """Split long text into chunks of max_chars."""
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunks.append(text[start:end])
        start = end
    return chunks


                      # AGENT 1: PDF READER 
def agent_pdf_reader():
    global pdf_text, pdf_path_global
    print("\n--- PDF READER AGENT ---")
    print("Example path (Kaggle): /kaggle/input/your-folder/your-file.pdf")
    path = input("Enter PDF file path: ").strip()
    if not path:
        print("❌ No path provided.")
        return

    try:
        text = read_pdf(path)
        if not text.strip():
            print("⚠ PDF loaded but no text extracted (maybe scanned/non-text PDF).")
        else:
            pdf_text = text
            pdf_path_global = path
            print(f"✅ PDF loaded successfully from: {path}")
            print(f"Approx length: {len(pdf_text)} characters")
    except Exception as e:
        print("❌ Error reading PDF:", e)


                            # AGENT 2: PDF SUMMARIZER 
def agent_pdf_summarizer():
    global pdf_text, pdf_path_global
    print("\n--- PDF SUMMARIZER AGENT ---")

    if not pdf_text or len(pdf_text.strip()) < 200:
        print("❌ PDF text is too short or unreadable.")
        return

    mode = input("Summary type? (short/medium/long) [default: medium]: ").strip().lower() or "medium"
    if mode == "short":
        chunks_limit = 1
        max_tokens = 350
    elif mode == "long":
        chunks_limit = 5
        max_tokens = 900
    else:
        chunks_limit = 2
        max_tokens = 550

    chunks = chunk_text(pdf_text, max_chars=4500)  # smaller chunks prevent safety block
    chunks = chunks[:chunks_limit]

    partial_summaries = []

    for i, ch in enumerate(chunks, 1):
        print(f"Processing chunk {i}/{len(chunks)} ...")

        if len(ch.strip()) < 50:
            print(f"⚠ Chunk {i} is too small, skipping.")
            continue

        prompt = f"""
Summarize the following academic text into bullet points and key concepts.
Focus on clarity, definitions, formulas, and important reasoning.
Avoid long paragraphs.

CONTENT:
{ch}
"""

        try:
            summary = llm(prompt, max_tokens=max_tokens)

            if not summary or summary.strip() == "":
                print(f"⚠ Chunk {i} returned empty. Skipping.")
                continue

            partial_summaries.append(summary)
            print(f"✅ Chunk {i} summarized")

        except Exception as e:
            print(f"❌ Error summarizing chunk {i}: {e}")
            continue

    if not partial_summaries:
        print("❌ No usable summaries were produced. Try summarizing manually or reduce chunk size.")
        return

    separator = "\n\n---\n\n".join(partial_summaries)

    final_prompt = f"""
Combine the following short summaries into a well-structured final set of study notes.
Use headings, bullet points, examples, and keep everything short and exam-focused.

TEXT:
{separator}
"""

    try:
        final_summary = llm(final_prompt, max_tokens=900)
        print("\n====== FINAL SHORT NOTES ======\n")
        print(final_summary)
        print("\n==============================\n")
    except Exception as e:
        print("❌ Failed final merge:", e)

                               # AGENT 3: MCQ GENERATOR 
def agent_mcq_generator():
    global pdf_text, pdf_path_global

    print("\n--- MCQ GENERATOR AGENT ---")

    if not pdf_text or len(pdf_text.strip()) < 100:
        print("❌ PDF text is empty or too short. Try another PDF or check if it is scanned.")
        return

    try:
        num_q = int(input("How many MCQs do you want? (e.g. 10): ").strip())
    except:
        num_q = 10

    # Reduce context to avoid filter issue
    base_text = pdf_text[:8000]

    prompt = f"""
You are an exam MCQ creator.

Generate {num_q} high-quality MCQs from the study material below.

Rules:
- Each question must have options A, B, C, D
- Clearly state: Answer: <option>
- No overly personal or unsafe content
- Focus on academic & safe concepts only

CONTENT:
{base_text}
"""

    try:
        response = llm(prompt, max_tokens=1000)

        if not response or len(response.strip()) == 0:
            print("⚠ Model returned empty result. Trying again with simpler prompt...")
            response = llm("Create " + str(num_q) + " academic multiple choice questions with answers.")

        print("\n====== GENERATED MCQs ======\n")
        print(response)
        print("\n============================\n")

    except Exception as e:
        print("❌ Model failed. Try summarizing first or reducing MCQ count.")
        print("Error:", e)

                                      # AGENT 4: STUDY PLAN GENERATOR 
def agent_study_plan():
    print("\n--- STUDY PLAN AGENT ---")
    exam_name = input("Exam name (e.g. DAA Sessional, Python Viva, etc.): ").strip() or "Your Exam"
    days_str = input("In how many days is your exam? (e.g. 7): ").strip()
    try:
        days_left = int(days_str)
    except:
        days_left = 7

    topics = input("List key topics (comma separated):\n(e.g. Arrays, Linked List, Trees, Graphs):\n").strip()
    weak = input("Which topics are you weak in? (comma separated, or leave blank): ").strip()
    daily_hours_str = input("How many hours per day can you study? (e.g. 3): ").strip()
    try:
        daily_hours = float(daily_hours_str)
    except:
        daily_hours = 3.0

    prompt = f"""
You are an AI Study Mentor.

Create a detailed {days_left}-day study plan for the exam: {exam_name}.

Information:
- Topics: {topics or "not specified"}
- Weak topics: {weak or "not specified"}
- Hours per day: {daily_hours}

Plan requirements:
- Day-wise schedule with time blocks
- More focus/time on weak topics
- Include revision days
- Include MCQ practice / previous year questions
- Use a clean, bullet/markdown format
- Very practical and realistic
"""

    try:
        plan = llm(prompt, max_tokens=1200)
        print("\n====== STUDY PLAN ======\n")
        print(plan)
        print("\n========================\n")
    except Exception as e:
        print("❌ Error generating study plan:", e)


                         # AGENT 5: DIAGRAM GENERATOR
def agent_diagram_generator():
    print("\n--- DIAGRAM GENERATOR AGENT ---")
    concept = input("Enter the concept for which you want a diagram (e.g. QuickSort, OS layers, CNN, Tree traversal): ").strip()
    if not concept:
        print("❌ No concept provided.")
        return

    style = input("Diagram style? (ascii/markdown/simple) [default: ascii]: ").strip().lower() or "ascii"

    prompt = f"""
You are a teaching assistant.

Create a clear {style} diagram to explain this concept to a student:
CONCEPT: {concept}

Requirements:
- Use only text (no images).
- Use boxes, arrows, indentation, or tree-style layouts.
- After the diagram, add 3–5 bullet points explaining the key idea.
"""

    try:
        diagram = llm(prompt, max_tokens=700)
        print("\n====== GENERATED DIAGRAM ======\n")
        print(diagram)
        print("\n================================\n")
    except Exception as e:
        print("❌ Error generating diagram:", e)


                           #ORCHESTRATOR / MAIN MENU 
def main_menu():
    print("====================================")
    print("    AI StudyMentor – Multi-Agent    ")
    print("====================================")

    while True:
        print("\nWhat do you want to do?")
        print("1. Load / Read a PDF")
        print("2. Summarize loaded PDF into short notes")
        print("3. Generate MCQs from loaded PDF")
        print("4. Create a personalized study plan")
        print("5. Generate a text-based diagram for a concept")
        print("0. Exit")

        choice = input("Enter your choice (0-5): ").strip()

        if choice == "1":
            agent_pdf_reader()
        elif choice == "2":
            agent_pdf_summarizer()
        elif choice == "3":
            agent_mcq_generator()
        elif choice == "4":
            agent_study_plan()
        elif choice == "5":
            agent_diagram_generator()
        elif choice == "0":
            print("👋 Exiting AI StudyMentor. All the best for your exam!")
            break
        else:
            print("❌ Invalid choice. Please select a number between 0 and 5.")


# ------------- RUN PROGRAM 
if __name__ == "__main__":
    main_menu()


In [ ]:
import os
for root, dirs, files in os.walk("/kaggle/input"):
    for file in files:
        print(os.path.join(root, file))


In [ ]:
model = genai.GenerativeModel("gemini-2.5-flash")
resp = model.generate_content("Hello!")
print(resp.text)
